In [1]:
import pandas as pd 
import re
from splinter import Browser
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pymongo

In [2]:
full_df = pd.read_csv('../Resources/lyric_df.csv')
lyrics_df = pd.read_csv('../Resources/english_lyrics.csv')


In [3]:
full_df

,Unnamed: 0,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,...,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,...,-4.718,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과..."
1,1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,...,-3.715,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and..."
2,2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,...,-4.280,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장..."
3,3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,...,-7.470,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my..."
4,4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,...,-7.381,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,...,-6.113,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...
454,454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,...,-6.449,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing
455,455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,...,-5.406,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya..."
456,456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,...,-7.769,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni..."


In [4]:
full_df = full_df.drop(['Unnamed: 0'], axis=1)

In [5]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,-4.718,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과..."
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,-3.715,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and..."
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,-4.280,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장..."
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,-7.470,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my..."
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,-7.381,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,-6.113,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,-6.449,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,-5.406,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya..."
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,-7.769,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni..."


# Clean/Process Lyrics

In [6]:
def remove_non_english(sentence):
    # Split the sentence into words
    words = sentence.split()
    
    # Use a regular expression to remove all non-alphabetic characters and keep only English words
    words = [re.sub(r'[^a-zA-Z]', ' ', word) for word in words]
    
    # Join the words back into a sentence
    sentence = ' '.join(words)
    
    return sentence

In [7]:
def remove_song_structure(lyrics):
    # Use a regular expression to remove the specific song structure elements
    lyrics = re.sub(r'\[(Pre-Chorus|Verse 1|Chorus|Intro|Verse|Verse 2|Outro|Bridge)\]', ' ', lyrics)
    
    return lyrics

In [8]:
def remove_extra_spaces(text):
    # Use a regular expression to remove multiple consecutive spaces
    text = re.sub(r' {2,}', ' ', text)
    
    return text

In [9]:
def remove_verse(lyrics):
    # Use a regular expression to remove the word "verse"
    lyrics = re.sub(r'Verse', '', lyrics, flags=re.IGNORECASE)
    
    return lyrics

In [10]:
full_df['filtered_lyrics'] = full_df['lyrics'].apply(remove_song_structure).apply(remove_non_english).apply(remove_extra_spaces).apply(remove_verse)

In [11]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,...,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,...,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,...,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,...,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,...,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,...,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...,Warren Hue I wanna be like I ve been stuck i...
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,...,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing,missing
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,...,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,...,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tru...


# Create missing song dataframe

In [12]:
missing_songs = full_df.loc[full_df['filtered_lyrics'] == 'missing']

In [13]:
missing_songs.head()

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,...,1,0.0264,0.00148,0.1670,0.249,135.917,214852,4,missing,missing
9,Life is a Bi…,Life is a Bi…,BIBI,60,BIBI,2021-04-28,4d79iPe88TzFewhFk2vrcy,0.722,0.501,10,...,0,0.0625,0.00000,0.0857,0.541,144.076,156742,4,missing,missing
11,비누,비누,BIBI,56,BIBI,2019-05-15,0lSJQip2wVidbu1JfZ4IEQ,0.842,0.522,4,...,0,0.0329,0.00000,0.1910,0.641,119.045,182200,4,missing,missing
12,Loveholic's hangover (feat. Sam Kim),Lowlife Princess: Noir,BIBI,57,Sam Kim,2022-11-18,2FmByq9VHuQ0N7A8KMmQCg,0.815,0.620,2,...,1,0.0520,0.04550,0.0916,0.214,103.975,168156,4,missing,missing
13,Dingo X BIBI - she got it,Dingo X BIBI - she got it,BIBI,56,BIBI,2020-08-06,29wNMwCoKetR1EdruKR3I9,0.757,0.687,7,...,0,0.0443,0.00000,0.1150,0.583,99.992,215736,4,missing,missing


In [14]:
missing_songs.reset_index(inplace=True)

In [15]:
missing_songs

,index,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,...,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing,missing
1,9,Life is a Bi…,Life is a Bi…,BIBI,60,BIBI,2021-04-28,4d79iPe88TzFewhFk2vrcy,0.722,0.501,...,0,0.0625,0.000000,0.0857,0.541,144.076,156742,4,missing,missing
2,11,비누,비누,BIBI,56,BIBI,2019-05-15,0lSJQip2wVidbu1JfZ4IEQ,0.842,0.522,...,0,0.0329,0.000000,0.1910,0.641,119.045,182200,4,missing,missing
3,12,Loveholic's hangover (feat. Sam Kim),Lowlife Princess: Noir,BIBI,57,Sam Kim,2022-11-18,2FmByq9VHuQ0N7A8KMmQCg,0.815,0.620,...,1,0.0520,0.045500,0.0916,0.214,103.975,168156,4,missing,missing
4,13,Dingo X BIBI - she got it,Dingo X BIBI - she got it,BIBI,56,BIBI,2020-08-06,29wNMwCoKetR1EdruKR3I9,0.757,0.687,...,0,0.0443,0.000000,0.1150,0.583,99.992,215736,4,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,444,458 gummy st.,SUGARTOWN,Warren Hue,32,Chasu,2019-12-13,68B9C88nDiFc88pnjlTctm,0.862,0.760,...,0,0.1830,0.000000,0.0792,0.448,150.050,183200,4,missing,missing
173,446,Getcho Mans,Brightside,Rich Brian,55,Warren Hue,2022-01-20,10b0n4KwuRyNteJ9WNjrdj,0.641,0.638,...,1,0.2140,0.000013,0.2200,0.313,174.929,211307,4,missing,missing
174,448,W (with yvngxchris),W (with yvngxchris),Warren Hue,38,yvngxchris,2022-03-25,4pFkivUXTiGcEQS4dfcVc6,0.719,0.504,...,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing,missing
175,452,IN MY BAG (with tobi lou),BOY OF THE YEAR,Warren Hue,54,tobi lou,2022-07-29,3Vdu1sTIUi45NZvguJDmhk,0.705,0.731,...,1,0.0430,0.000000,0.2780,0.893,75.504,219676,4,missing,missing


In [16]:
missing_songs = missing_songs.drop(['index'], axis=1)

In [17]:
missing_songs

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,...,1,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing,missing
1,Life is a Bi…,Life is a Bi…,BIBI,60,BIBI,2021-04-28,4d79iPe88TzFewhFk2vrcy,0.722,0.501,10,...,0,0.0625,0.000000,0.0857,0.541,144.076,156742,4,missing,missing
2,비누,비누,BIBI,56,BIBI,2019-05-15,0lSJQip2wVidbu1JfZ4IEQ,0.842,0.522,4,...,0,0.0329,0.000000,0.1910,0.641,119.045,182200,4,missing,missing
3,Loveholic's hangover (feat. Sam Kim),Lowlife Princess: Noir,BIBI,57,Sam Kim,2022-11-18,2FmByq9VHuQ0N7A8KMmQCg,0.815,0.620,2,...,1,0.0520,0.045500,0.0916,0.214,103.975,168156,4,missing,missing
4,Dingo X BIBI - she got it,Dingo X BIBI - she got it,BIBI,56,BIBI,2020-08-06,29wNMwCoKetR1EdruKR3I9,0.757,0.687,7,...,0,0.0443,0.000000,0.1150,0.583,99.992,215736,4,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,458 gummy st.,SUGARTOWN,Warren Hue,32,Chasu,2019-12-13,68B9C88nDiFc88pnjlTctm,0.862,0.760,4,...,0,0.1830,0.000000,0.0792,0.448,150.050,183200,4,missing,missing
173,Getcho Mans,Brightside,Rich Brian,55,Warren Hue,2022-01-20,10b0n4KwuRyNteJ9WNjrdj,0.641,0.638,1,...,1,0.2140,0.000013,0.2200,0.313,174.929,211307,4,missing,missing
174,W (with yvngxchris),W (with yvngxchris),Warren Hue,38,yvngxchris,2022-03-25,4pFkivUXTiGcEQS4dfcVc6,0.719,0.504,6,...,1,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing,missing
175,IN MY BAG (with tobi lou),BOY OF THE YEAR,Warren Hue,54,tobi lou,2022-07-29,3Vdu1sTIUi45NZvguJDmhk,0.705,0.731,3,...,1,0.0430,0.000000,0.2780,0.893,75.504,219676,4,missing,missing


In [18]:
missing_songs.to_csv('../Resources/missing_songs.csv', sep=',')

# Create found lyrics dataframe

In [19]:
songs_df = full_df.loc[full_df['filtered_lyrics'] != 'missing']

In [20]:
songs_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,...,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,...,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,...,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,...,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...
5,Restless,LISTEN 035 Restless,BIBI,62,BIBI,2020-02-02,4AHNi316rl3KI3hKIsdxZe,0.575,0.607,5,...,1,0.0274,0.000112,0.1150,0.298,89.985,267080,4,"[비비 ""신경쓰여"" 가사][Verse 1]불 꺼진 방안 뒹굴다전화는 왜 잠만 잘 자...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,THIRTYNINE,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,2qr71lIuzyme9oVwYngfyM,0.587,0.645,7,...,1,0.1340,0.000000,0.1700,0.708,95.061,104985,4,"[Verse 1]Ayy, ayy, ayyWent to Los AngelesDissi...",Ayy ayy ayyWent to Los AngelesDissin all these...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,...,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...,Warren Hue I wanna be like I ve been stuck i...
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,...,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,...,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tru...


In [21]:
songs_df.reset_index(inplace=True)

In [22]:
songs_df.head(10)

,index,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,...,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...
1,1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,...,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...
2,2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,...,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...
3,3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,...,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...
4,5,Restless,LISTEN 035 Restless,BIBI,62,BIBI,2020-02-02,4AHNi316rl3KI3hKIsdxZe,0.575,0.607,...,1,0.0274,0.000112,0.1150,0.298,89.985,267080,4,"[비비 ""신경쓰여"" 가사][Verse 1]불 꺼진 방안 뒹굴다전화는 왜 잠만 잘 자...",
5,6,MotoSpeed 24,Sweet Sorrow of Mother,BIBI,59,BIBI,2022-10-24,0BPwEuU9qegqVSob1Ihm97,0.610,0.597,...,1,0.0294,0.000018,0.1360,0.295,95.004,133905,4,"[비비 ""모토스피드 24시"" 가사][Verse 1]밤 만이 계속되는 이 심플한 도시...",I like that I like that Let s ride Oh oh oh o...
6,7,Maybe if,Our Beloved Summer (Original Television Soundt...,BIBI,59,BIBI,2021-12-13,6MxIazgQENgaa6StpFxGpu,0.593,0.437,...,1,0.0552,0.000000,0.0919,0.484,121.662,209600,4,"[비비 ""우리가 헤어져야 했던 이유"" 가사][Verse 1]Maybe If IWok...",Maybe If IWoke up in the morningHearing your ...
7,8,JOTTO,Lowlife Princess: Noir,BIBI,58,BIBI,2022-11-18,2hP0BkoawxEbnT78BlcILq,0.701,0.755,...,1,0.0929,0.000000,0.2430,0.488,105.005,193856,4,"[비비 ""조또"" 가사][Verse 1]뒤도 돌아보지 않고 떠나더니옷깃 부여잡는 손을...",Jotto mm mmJotto mm mm mmJotto Jotto pills an...
8,10,PADO,PADO,BIBI,56,BIBI,2021-09-10,1PSDpiTgnkxHc7GhKrHCaA,0.690,0.784,...,1,0.0324,0.000000,0.1520,0.642,120.101,193133,4,"[비비 ""PADO"" 가사][Verse 1]너는 두 눈을 감은 채내게 기대어오며머리칼...",PADO If someone hear me call me You are my wa...
9,16,Witch Hunt,Lowlife Princess: Noir,BIBI,57,BIBI,2022-11-18,51C2Xp8avmQpLwJWqt3YJK,0.763,0.698,...,1,0.0587,0.000003,0.0947,0.708,140.012,104697,4,"[비비 ""마녀사냥"" 가사][Intro]I need you[Chorus]When my...",I need you When my instinct hit me badCryin l...


In [23]:
songs_df = songs_df.drop(['index'], axis=1)

In [24]:
songs_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,...,0,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,...,0,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,...,0,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,...,0,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...
4,Restless,LISTEN 035 Restless,BIBI,62,BIBI,2020-02-02,4AHNi316rl3KI3hKIsdxZe,0.575,0.607,5,...,1,0.0274,0.000112,0.1150,0.298,89.985,267080,4,"[비비 ""신경쓰여"" 가사][Verse 1]불 꺼진 방안 뒹굴다전화는 왜 잠만 잘 자...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,THIRTYNINE,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,2qr71lIuzyme9oVwYngfyM,0.587,0.645,7,...,1,0.1340,0.000000,0.1700,0.708,95.061,104985,4,"[Verse 1]Ayy, ayy, ayyWent to Los AngelesDissi...",Ayy ayy ayyWent to Los AngelesDissin all these...
277,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,...,0,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...,Warren Hue I wanna be like I ve been stuck i...
278,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,...,1,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...
279,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,...,1,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tru...


In [25]:
songs_df.to_csv('../Resources/full_songs_df.csv', sep=',')

In [26]:
sample_df = songs_df[['track','artist']].copy()

# Create a df copy of two argument columns to pass into genre function below

In [27]:
sample_df

,track,artist
0,BIBI Vengeance,BIBI
1,KAZINO,BIBI
2,Animal Farm,BIBI
3,BAD SAD AND MAD,BIBI
4,Restless,BIBI
...,...,...
276,THIRTYNINE,Warren Hue
277,HANDSOME,Warren Hue
278,DEMOSTAR BEENLIT,Warren Hue
279,RUNAWAY W ME,Warren Hue


In [28]:
sample_df.insert(loc=2, column='genre', value='')

In [38]:
sample_df.to_csv('../Resources/sample_df.csv', sep=',')

In [39]:
sample_df

,track,artist,genre
0,BIBI Vengeance,BIBI,
1,KAZINO,BIBI,
2,Animal Farm,BIBI,
3,BAD SAD AND MAD,BIBI,
4,Restless,BIBI,
...,...,...,...
276,THIRTYNINE,Warren Hue,
277,HANDSOME,Warren Hue,
278,DEMOSTAR BEENLIT,Warren Hue,
279,RUNAWAY W ME,Warren Hue,


In [51]:
# #Less optimal function to scrape genre tag from genius
# def scrape_genres(df):
    
#     executable_path = {'executable_path': ChromeDriverManager().install()}
#     browser = Browser('chrome', **executable_path, headless=False)

#     for index, row in df.iterrows():

#         artist = row['artist'].replace(' ','-') 
#         song = row['track'].replace(' ','-')

#         url = 'https://genius.com/'+ artist + '-' + song + '-' + 'lyrics'
#         browser.visit(url)

#         time.sleep(1)
#         # Scrape page into Soup
#         html = browser.html
#         soup = BeautifulSoup(html, 'html.parser')

#         div_class = soup.find_all('div', class_='SongTags__Container-xixwg3-1 bZsZHM')
        
#         genre_data = []
#         for r in div_class:
#             try:
#                 anchor = r.find_all('a')

#                 for a in anchor:
#                     genre = a.text
#                     genre_data.append(genre)
#                     time.sleep(1)
                    
#             except (KeyError, IndexError):
#                 print("Missing field/result... skipping.")
        
#         # Concatenate all values in genre_data into a single string
#         genre_string = ', '.join(genre_data)
        
#         # Assign the genre string to a single cell in the dataframe
#         df.loc[index, 'genres'] = genre_string

#     return df

In [34]:
# #function to scrape for singel genre tag from genius
# def scrape_genres(artist,song):

#     url = f'https://genius.com/{artist}-{song}-lyrics'
    
#     # Make a request to the website and parse the HTML response
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     div_class = soup.find_all('div', class_="SongTags__Container-xixwg3-1 bZsZHM")
    
#     for r in div_class:
#         anchor = r.find_all('a')
#         for a in anchor:
#             genre=a.text
    
#     # Return the genre tags as a list
#     return genre          

In [35]:
# genre_data = []

# for index, row in sample_df.iterrows():

#     artist = row['artist'].replace(' ','-') 
#     song = row['track'].replace(' ','-')

#     genre_tags = scrape_genres(artist,song)
    
#     # Append the genre tags to the list
#     genre_data.append(genre_tags)

# # Add the genre_tags_list to the dataframe as a new column 'genre_tags'
# sample_df['genres'] = genre_data          

In [ ]:
# sample_df

In [ ]:
# sample_df.to_csv('../Resources/sample_df.csv', sep=',')

In [58]:
test_df = pd.read_csv('../Resources/sample_df.csv', usecols=["track", "artist"])

df = pd.read_csv("../Resources/sample_df.csv", usecols=["track", "artist"])

# Load a test dataframe for trial

In [59]:
test_df

,track,artist
0,BIBI Vengeance,BIBI
1,KAZINO,BIBI
2,Animal Farm,BIBI
3,BAD SAD AND MAD,BIBI
4,Restless,BIBI
...,...,...
276,THIRTYNINE,Warren Hue
277,HANDSOME,Warren Hue
278,DEMOSTAR BEENLIT,Warren Hue
279,RUNAWAY W ME,Warren Hue


In [ ]:
# test = scrape_genres(test_df)
#35 mins to run

In [ ]:
# test 

In [50]:
# test.to_csv('../Resources/trial_one_genre.csv',sep=',')

# Create a scrape genre function

In [84]:
#Optimal function to scrape genre tag from genius
def scrape_genre(artist,song):

    url = f'https://genius.com/{artist}-{song}-lyrics'
    
    # Make a request to the website and parse the HTML response
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    div_class = soup.find_all('div', class_="SongTags__Container-xixwg3-1 bZsZHM")
    
    if div_class:
        genre_tags = []
        for r in div_class:
            anchor = r.find_all('a')
            for a in anchor:
                genre_tags.append(a.text)
        return genre_tags
    else:
        return []

In [61]:
data = []

for index, row in test_df.iterrows():

    artist = row['artist'].replace(' ','-') 
    song = row['track'].replace(' ','-')

    genre_tags = scrape_genre(artist,song)
    
    # Append the genre tags to the list
    data.append(genre_tags)

# Add the genre_tags_list to the dataframe as a new column 'genre_tags'
test_df['genre'] = data

In [62]:
test_df

,track,artist,genre
0,BIBI Vengeance,BIBI,"[R&B, K-R&B, Reggaetón, South Korea, Genius Ko..."
1,KAZINO,BIBI,"[R&B, South Korea, Trap, Korean]"
2,Animal Farm,BIBI,"[R&B, Korean, K-R&B, K-Solo, South Korea, Geni..."
3,BAD SAD AND MAD,BIBI,"[R&B, K-R&B, Korean, K-Solo, South Korea, Geni..."
4,Restless,BIBI,"[R&B, South Korea, K-R&B, Korean]"
...,...,...,...
276,THIRTYNINE,Warren Hue,"[Rap, Indonesia]"
277,HANDSOME,Warren Hue,"[Rap, Indonesia]"
278,DEMOSTAR BEENLIT,Warren Hue,[Rap]
279,RUNAWAY W ME,Warren Hue,"[Pop, Indonesia]"


In [87]:
data = []

for index, row in full_df.iterrows():

    artist = row['artist'].replace(' ','-') 
    song = row['track'].replace(' ','-')

    genre_tags = scrape_genre(artist,song)
    
    # Append the genre tags to the list
    data.append(genre_tags)

# Add the genre_tags_list to the dataframe as a new column 'genre_tags'
full_df['genre'] = data

In [89]:
full_df

,track,album,artist,popularity,featured,release_date,track_id,danceability,energy,key,...,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lyrics,filtered_lyrics,genre
0,BIBI Vengeance,Lowlife Princess: Noir,BIBI,76,BIBI,2022-11-17,0ra3bPUOj2YnY4FJHXtgHZ,0.442,0.748,11,...,0.0895,0.000128,0.0642,0.408,99.981,165135,4,"[비비 ""나쁜년"" 가사][Verse 1]들어오라 해눈을 깔어 왜비밀로 할게네 부정과...",Watch me danceWatch me sing Watch me danceWat...,"[R&B, K-R&B, Reggaetón, South Korea, Genius Ko..."
1,KAZINO,KAZINO,BIBI,68,BIBI,2020-04-29,245onUPHGD1DYuiacxTuW0,0.625,0.686,6,...,0.0414,0.000000,0.1100,0.263,133.041,185947,4,"[비비 ""사장님 도박은 재미로 하셔야 합니다"" 가사][Verse 1]Dice and...",Dice and Ice cube Home girls bitches Spin it ...,"[R&B, South Korea, Trap, Korean]"
2,Animal Farm,Animal Farm,BIBI,67,BIBI,2022-09-27,3gAXVQOR0h7Ks71lsr5MFZ,0.617,0.570,5,...,0.0308,0.000006,0.5500,0.375,107.971,203946,3,"[비비 ""가면무도회"" 가사][Verse 1]죄악의 단어위선의 가면넘쳐흐르는 무도회장...",champagne It might be pretty it might be beau...,"[R&B, Korean, K-R&B, K-Solo, South Korea, Geni..."
3,BAD SAD AND MAD,Life is a Bi…,BIBI,66,BIBI,2021-04-28,0zPbUxvYRlCGwruakkXPLI,0.746,0.558,0,...,0.0880,0.000190,0.0904,0.638,90.013,154988,4,"[비비 ""BAD SAD AND MAD"" 가사][Refrain]You blued my...",BAD SAD AND MAD Refrain You blued my bruisePi...,"[R&B, K-R&B, Korean, K-Solo, South Korea, Geni..."
4,"Very, Slowly",Twenty-Five Twenty-One OST Part 3,BIBI,62,BIBI,2022-02-20,7GkHIsnziYgk6j1lx2TK6H,0.528,0.423,8,...,0.0264,0.001480,0.1670,0.249,135.917,214852,4,missing,missing,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,HANDSOME,BOY OF THE YEAR,Warren Hue,45,Warren Hue,2022-07-29,58z1hY9hrwliXRIvQe8UOX,0.762,0.686,10,...,0.1760,0.000000,0.0846,0.385,89.992,126962,4,[Verse 1: Warren Hue](I wanna be like)I've bee...,Warren Hue I wanna be like I ve been stuck i...,"[Rap, Indonesia]"
454,W (with yvngxchris),BOY OF THE YEAR,Warren Hue,39,yvngxchris,2022-07-29,4U2BfAHYdfDG25XL9eG0BO,0.719,0.504,6,...,0.4220,0.000000,0.2150,0.493,164.926,151446,4,missing,missing,[]
455,DEMOSTAR BEENLIT,BOY OF THE YEAR,Warren Hue,39,Warren Hue,2022-07-29,647TFpvzp7uPAAvOfetag9,0.779,0.871,1,...,0.2900,0.000003,0.0995,0.532,162.055,126043,4,"[Intro]Uh, uh, ayy, ya-yaUh, uh, turn-turn, ya...",Uh uh ayy ya yaUh uh turn turn ya yaT turn ya ...,[Rap]
456,RUNAWAY W ME,BOY OF THE YEAR,Warren Hue,42,Warren Hue,2022-07-29,3EXg5PZgK1dwZyzqLp9qES,0.651,0.677,4,...,0.1380,0.000000,0.2470,0.324,129.923,181840,4,"[Chorus]Won't you, run away with me this eveni...",Won t you run away with me this evening Oh tru...,"[Pop, Indonesia]"


In [90]:
songs_df.to_csv('../Test-Data/pre-final-88-genre.csv', sep=',')

In [63]:
test_df.to_csv('../Resources/trial_run_genre.csv', sep=',')